# capstone project notebook 2 

### This notebook will focus on  model training and model evaluation.

In [24]:
#import required libraries for handling and visualizing data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#more libraries from the sklearn package will be imported later for modeling and evaluation
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#the format for both columns is 'object' and not the datetime type that we would like and there are issues with the timestamp
#this may be and issue, but lets leave for now

#lets look at the volume of incidents overtime... first we need to convert the time to the right format.
#lets use the INCDTTM column since that atleast captures the time for some records
dt['DATETIME']=dt['INCDTTM'].astype('datetime64[ns]')
print(dt['DATETIME'].dtypes)
print(dt['DATETIME'].describe())
dt[['DATETIME']].head(15)

In [25]:
df_exp.to_csv('explored_features.csv', index=False)

In [27]:
#identified as possible model features... lets import that and pick up our data cleaning efforts there...
df_exp = pd.read_csv('explored_features.csv')
df_exp['DATETIME']=df_exp['DATETIME'].astype('datetime64[ns]')
df_exp.head()

,DATETIME,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,SEVERITYCODE
0,2013-03-27 14:54:00,Angles,2,0,0,2,N,N,Overcast,Wet,Daylight,N,2
1,2006-12-20 18:55:00,Sideswipe,2,0,0,2,N,0,Raining,Wet,Dark - Street Lights On,N,1
2,2004-11-18 10:20:00,Parked Car,4,0,0,3,N,0,Overcast,Dry,Daylight,N,1
3,2013-03-29 09:26:00,Other,3,0,0,3,N,N,Clear,Dry,Daylight,N,1
4,2004-01-28 08:04:00,Angles,2,0,0,2,N,0,Raining,Wet,Daylight,N,2


In [28]:
df = df_exp[['DATETIME', 'COLLISIONTYPE','PERSONCOUNT','VEHCOUNT','INATTENTIONIND','UNDERINFL','SPEEDING','SEVERITYCODE']].copy()
print(df.shape)
df.head()

(194673, 8)


,DATETIME,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,SPEEDING,SEVERITYCODE
0,2013-03-27 14:54:00,Angles,2,2,N,N,N,2
1,2006-12-20 18:55:00,Sideswipe,2,2,N,0,N,1
2,2004-11-18 10:20:00,Parked Car,4,3,N,0,N,1
3,2013-03-29 09:26:00,Other,3,3,N,N,N,1
4,2004-01-28 08:04:00,Angles,2,2,N,0,N,2


In [30]:

#lets check how many blank values there are in each column:
df.isnull().sum(axis=0)
df.dropna(axis=0,subset=['COLLISIONTYPE'],inplace=True)
df.isnull().sum(axis=0)
df.dropna(axis=0,subset=['UNDERINFL'],inplace=True)
print(df.shape)
df.isnull().sum(axis=0)
df['PERSONCOUNT'].value_counts()
#what is the VEHCOUNT distribution when the PERSONCOUNT = 0
df[df['PERSONCOUNT']==0]['VEHCOUNT'].value_counts()

(189768, 8)


2     4267
1      771
3      390
4       80
5       19
6        5
0        5
7        2
11       1
9        1
Name: VEHCOUNT, dtype: int64

In [31]:
#looking at severity proportion (injury vs property) of collisions before and after removing samples where PERSONCOUNT = 0
before = (df['SEVERITYCODE'].value_counts()[2]) / (df['SEVERITYCODE'].value_counts()[1])
after = (df[df['PERSONCOUNT']!=0]['SEVERITYCODE'].value_counts()[2])/(df[df['PERSONCOUNT']!=0]['SEVERITYCODE'].value_counts()[1])

print('Ratio of injury to property collisions for all samples (BEFORE): '+'{:.2f}'.format(before))
print('Ratio of injury to property collisions without PERSONCOUNT=0 samples (AFTER): '+'{:.2f}'.format(after))

Ratio of injury to property collisions for all samples (BEFORE): 0.43
Ratio of injury to property collisions without PERSONCOUNT=0 samples (AFTER): 0.43


In [32]:
df = df[df['PERSONCOUNT']!=0]
print(df.shape)

(184227, 8)


In [33]:
#convert N to 0, Y to 1 in INATTENTIONIND, UNDERINFL and SPEEDING columns
df['SPEEDING'].replace(('Y','N'),(1,0),inplace=True)
df['INATTENTIONIND'].replace(('Y','N'),(1,0),inplace=True)
df['UNDERINFL'].replace(('Y','N'),(1,0),inplace=True)
df.head()

,DATETIME,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,SPEEDING,SEVERITYCODE
0,2013-03-27 14:54:00,Angles,2,2,0,0,0,2
1,2006-12-20 18:55:00,Sideswipe,2,2,0,0,0,1
2,2004-11-18 10:20:00,Parked Car,4,3,0,0,0,1
3,2013-03-29 09:26:00,Other,3,3,0,0,0,1
4,2004-01-28 08:04:00,Angles,2,2,0,0,0,2


In [34]:
#the existing 1s and 0s in the UNDERINFL column are strings and need to be converted to integers as well
df['UNDERINFL'].replace(('0','1'),(0,1),inplace=True)
df['UNDERINFL'].value_counts()

0    175107
1      9120
Name: UNDERINFL, dtype: int64

In [35]:
#add columns: PCOUNT_OVER2, VCOUNT_IS2 to track whether there are 3 or more people and whether there are 2 vehicles or other
df['PCOUNT_OVER2'] = [1 if b>=3 else 0 for b in df['PERSONCOUNT']]
df['VCOUNT_IS2'] = [1 if b==2 else 0 for b in df['VEHCOUNT']]
df.head()

,DATETIME,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,SPEEDING,SEVERITYCODE,PCOUNT_OVER2,VCOUNT_IS2
0,2013-03-27 14:54:00,Angles,2,2,0,0,0,2,0,1
1,2006-12-20 18:55:00,Sideswipe,2,2,0,0,0,1,0,1
2,2004-11-18 10:20:00,Parked Car,4,3,0,0,0,1,1,0
3,2013-03-29 09:26:00,Other,3,3,0,0,0,1,1,0
4,2004-01-28 08:04:00,Angles,2,2,0,0,0,2,0,1


In [36]:
#add column 'RUSHHOUR' to identify whether time is during the rushhour period on a weekday
wkdays = [0,1,2,3,4] #list of values that are weekdays
rushhrs = [5,6,7,8,15,16,17,18] #list of hours that are during AM/PM rushhour periouds (5-8 and 3-6)

df['RUSHHOUR'] = [1 if (b.hour in rushhrs) & (b.dayofweek in wkdays) else 0 for b in df.DATETIME]
df.head()

,DATETIME,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,SPEEDING,SEVERITYCODE,PCOUNT_OVER2,VCOUNT_IS2,RUSHHOUR
0,2013-03-27 14:54:00,Angles,2,2,0,0,0,2,0,1,0
1,2006-12-20 18:55:00,Sideswipe,2,2,0,0,0,1,0,1,1
2,2004-11-18 10:20:00,Parked Car,4,3,0,0,0,1,1,0,0
3,2013-03-29 09:26:00,Other,3,3,0,0,0,1,1,0,0
4,2004-01-28 08:04:00,Angles,2,2,0,0,0,2,0,1,1


In [37]:
#when first observing the dataset, the INCDTTM column had some dates that were missing a timestamp.. these could be
#incorrectly classified as non-rushhour, since we don't know the true times.. lets determine the number of impacted items
#and remove them

#first lets look at an example... in the first 10 rows of the dataframe, notice row 6, which has a timestamp of 00:00:00
#this is because the original INCDTTM column had only a date, so the timestamp defaults to 00:00:00 when converted to
#the datetime format
df.head(10)

,DATETIME,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,SPEEDING,SEVERITYCODE,PCOUNT_OVER2,VCOUNT_IS2,RUSHHOUR
0,2013-03-27 14:54:00,Angles,2,2,0,0,0,2,0,1,0
1,2006-12-20 18:55:00,Sideswipe,2,2,0,0,0,1,0,1,1
2,2004-11-18 10:20:00,Parked Car,4,3,0,0,0,1,1,0,0
3,2013-03-29 09:26:00,Other,3,3,0,0,0,1,1,0,0
4,2004-01-28 08:04:00,Angles,2,2,0,0,0,2,0,1,1
5,2019-04-20 17:42:00,Angles,2,2,0,0,0,1,0,1,0
6,2008-12-09 00:00:00,Angles,2,2,0,0,0,1,0,1,0
7,2020-04-15 17:47:00,Cycles,3,1,0,0,0,2,1,0,1
8,2006-06-15 13:00:00,Parked Car,2,2,0,0,0,1,0,1,0
9,2006-03-20 15:49:00,Angles,2,2,0,0,0,2,0,1,1


In [38]:
#lets remove these samples... first we need to set the index to the DATETIME column, then we can make use of the
#between_times method
df.set_index('DATETIME',inplace=True)
df.head()

,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,SPEEDING,SEVERITYCODE,PCOUNT_OVER2,VCOUNT_IS2,RUSHHOUR
DATETIME,,,,,,,,,,
2013-03-27 14:54:00,Angles,2,2,0,0,0,2,0,1,0
2006-12-20 18:55:00,Sideswipe,2,2,0,0,0,1,0,1,1
2004-11-18 10:20:00,Parked Car,4,3,0,0,0,1,1,0,0
2013-03-29 09:26:00,Other,3,3,0,0,0,1,1,0,0
2004-01-28 08:04:00,Angles,2,2,0,0,0,2,0,1,1


In [39]:
#now lets create a new df with the samples that fall between a time of 00:00:00 and 00:00:01
#this represents midnight and one second after midnight, and since the timestamp values only have hours and minutes
#it should only capture those samples where not time originally existed and the timestamp defaulted to 00:00:00
df2 = df.between_time('00:00:00','00:00:01')
print(df2.shape)
df2.head()

(25526, 10)


,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,SPEEDING,SEVERITYCODE,PCOUNT_OVER2,VCOUNT_IS2,RUSHHOUR
DATETIME,,,,,,,,,,
2008-12-09,Angles,2,2,0,0,0,1,0,1,0
2005-07-01,Left Turn,3,2,0,0,0,2,1,1,0
2009-06-01,Parked Car,2,2,0,0,0,1,0,1,0
2005-11-05,Left Turn,3,2,0,0,0,1,1,1,0
2006-08-05,Angles,2,2,0,0,0,1,0,1,0


In [40]:
#by spotchecking the original dataset, we can confirm that the 4 records above were originally lacking a time value
#lets overwrite the original dataframe to remove the 25526 rows in df2, where the timestamp is 00:00:00
#this can be done by swapping the start and end time values..

df = df.between_time('00:00:01','23:59:59')
#to check, the number of records should equal the previous size (184227) minus the size of df2 (25526) leaving 158701
print(df.shape)
#also notice how the old sample in row 6 has been removed and the record now has a timevalue
df.reset_index(inplace=True)
df.head(10)

(158701, 10)


,DATETIME,COLLISIONTYPE,PERSONCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,SPEEDING,SEVERITYCODE,PCOUNT_OVER2,VCOUNT_IS2,RUSHHOUR
0,2013-03-27 14:54:00,Angles,2,2,0,0,0,2,0,1,0
1,2006-12-20 18:55:00,Sideswipe,2,2,0,0,0,1,0,1,1
2,2004-11-18 10:20:00,Parked Car,4,3,0,0,0,1,1,0,0
3,2013-03-29 09:26:00,Other,3,3,0,0,0,1,1,0,0
4,2004-01-28 08:04:00,Angles,2,2,0,0,0,2,0,1,1
5,2019-04-20 17:42:00,Angles,2,2,0,0,0,1,0,1,0
6,2020-04-15 17:47:00,Cycles,3,1,0,0,0,2,1,0,1
7,2006-06-15 13:00:00,Parked Car,2,2,0,0,0,1,0,1,0
8,2006-03-20 15:49:00,Angles,2,2,0,0,0,2,0,1,1
9,2006-05-25 18:20:00,Other,2,2,0,0,0,1,0,1,1


In [41]:
#lets check what the distribution of collision severity was for each of the subsets...
#for the removed samples in df2
df2.reset_index(inplace=True)
print('Value counts for removed samples: \n',df2['SEVERITYCODE'].value_counts())
print('Ratio of injury to property collisions for removed samples: \n',(df2['SEVERITYCODE'].value_counts()[2])/(df2['SEVERITYCODE'].value_counts()[1]))
#for the remaining samples in the df
print('Value counts for remaining samples: \n',df['SEVERITYCODE'].value_counts())
print('Ratio of injury to property collisions for remaining samples: \n',(df['SEVERITYCODE'].value_counts()[2])/(df['SEVERITYCODE'].value_counts()[1]))

Value counts for removed samples: 
 1    18193
2     7333
Name: SEVERITYCODE, dtype: int64
Ratio of injury to property collisions for removed samples: 
 0.4030671137250591
Value counts for remaining samples: 
 1    110649
2     48052
Name: SEVERITYCODE, dtype: int64
Ratio of injury to property collisions for remaining samples: 
 0.4342741461739374


In [42]:
#lets look at the distribution of the removed samples by year...
df2.groupby(df2.DATETIME.dt.year).count()['SEVERITYCODE']

DATETIME
2004    3583
2005    4439
2006    4598
2007    4335
2008    4073
2009    3399
2010     126
2011     626
2012      24
2013      65
2014      54
2015      38
2016      62
2017      31
2018      45
2019      23
2020       5
Name: SEVERITYCODE, dtype: int64

In [43]:
#lets look at the distribution for the remaining samples
df.groupby(df.DATETIME.dt.year).count()['SEVERITYCODE']

DATETIME
2004     8165
2005    10594
2006    10559
2007    10119
2008     9587
2009     8335
2010    10664
2011    10286
2012    10201
2013     9881
2014    11470
2015    10411
2016    10524
2017     8065
2018     9854
2019     8730
2020     1256
Name: SEVERITYCODE, dtype: int64

### Observations:
the ratio of injury to property collisions is slightly higher for the remaining samples, however,
it is still in line with the overall ratio of 0.43 injury collisions for every 1 property collision
the removed samples come overwhelmingly from the years 2004-2009; perhaps historically the time of incident wasn't consistently recorded
there is still ~160k samples remaining in the dataset, which could be used for training/testing

In [45]:
#Then we can compile the final feature set.

features_df = df[['SEVERITYCODE','RUSHHOUR','PCOUNT_OVER2','VCOUNT_IS2','INATTENTIONIND','UNDERINFL','SPEEDING']]
features_df = pd.concat([features_df,pd.get_dummies(df['COLLISIONTYPE'])], axis=1)
print(features_df.shape)
features_df.head()

(158701, 17)


,SEVERITYCODE,RUSHHOUR,PCOUNT_OVER2,VCOUNT_IS2,INATTENTIONIND,UNDERINFL,SPEEDING,Angles,Cycles,Head On,Left Turn,Other,Parked Car,Pedestrian,Rear Ended,Right Turn,Sideswipe
0,2,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,2,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0


## 4.0 Balancing
Since there are more than 2x the number of property collisions (majority class) as there are injury collisions (minority class), the dataset should be balanced to avoid introducing bias to any models.

There are a number of ways to do this, including:

up-sampling the monority class
down-sampling the majority class
using other methods (Decision trees) or metrics (Area Under ROC Curve or Penalization Algorithms)
For this project, we will use a number of these methods.

The model will also be trained using a number of different methods, including a Decision Tree, in order to find the most effective model.
Area under the curve will also be used as an evaluation metric, in addition to accuracy score, jaccard index, f1 and logloss

In [46]:
#lets observe how unbalanced the dataset is...
features_df['SEVERITYCODE'].value_counts()

1    110649
2     48052
Name: SEVERITYCODE, dtype: int64

In [47]:
from sklearn.utils import resample

In [48]:
#create dataframes of the majority and minority classes
df_maj = features_df[features_df['SEVERITYCODE']==1]
df_min = features_df[features_df['SEVERITYCODE']==2]

print('Size of majority class: ', df_maj.shape)
print('Size of minority class: ', df_min.shape)

Size of majority class:  (110649, 17)
Size of minority class:  (48052, 17)


In [49]:
#downsample the majority class
#the number of samples (n_samples) is set to 48052 to match the minority class size
df_maj_down = resample(df_maj,replace=False,n_samples=48052,random_state=123)

df_bal = pd.concat([df_maj_down,df_min])

df_bal['SEVERITYCODE'].value_counts()

2    48052
1    48052
Name: SEVERITYCODE, dtype: int64

In [50]:
df_bal.reset_index(drop=True,inplace=True)
df_bal.head()

,SEVERITYCODE,RUSHHOUR,PCOUNT_OVER2,VCOUNT_IS2,INATTENTIONIND,UNDERINFL,SPEEDING,Angles,Cycles,Head On,Left Turn,Other,Parked Car,Pedestrian,Rear Ended,Right Turn,Sideswipe
0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0


In [53]:
df_bal.to_csv('balance.csv', index=False)